In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

In [ ]:
mydf = spark.read.parquet("s3://zandras3/502final/df.parquet/*")

In [ ]:
mydf.printSchema()

In [ ]:
mydf.show(5)